# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [11]:
pip install geoviews  


  Using cached geoviews-1.10.0-py2.py3-none-any.whl (504 kB)
  Using cached Cartopy-0.21.1.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached shapely-2.0.1-cp310-cp310-win_amd64.whl (1.4 MB)
  Using cached pyproj-3.6.0-cp310-cp310-win_amd64.whl (5.7 MB)
  Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
Failed to build cartopy


  error: subprocess-exited-with-error
  
  × Building wheel for cartopy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [279 lines of output]
      <string>:90: UserWarning: Unable to determine GEOS version. Ensure you have 3.7.2 or later installed, or installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\crs.py -> build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\geodesic.py -> build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\img_transform.py -> build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\util.py -> build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\vector_transform.py -> build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\_epsg.py -> build\lib.win-amd64-cpython-310\cartopy
      copying lib\cartopy\_

In [12]:
pip install plotly

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,293.58,76,100,1.79,PT,1689543428
1,1,port saint john's,-31.6229,29.5448,289.58,60,20,3.09,ZA,1689543428
2,2,ust'-kulom,61.6864,53.6902,287.01,71,100,0.27,RU,1689543428
3,3,port-aux-francais,-49.3500,70.2167,275.05,54,82,10.25,TF,1689543428
4,4,gorzyce,50.6672,21.8401,293.53,85,7,3.24,PL,1689543429


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
# Configure the map plot
import plotly.express as px

# Create a scatter map plot
fig = px.scatter_mapbox(city_data_df, lat='Lat', lon='Lng', hover_name='City', size='Humidity',
                        color='Humidity', zoom=3)

# Customize the map layout
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})

# Display the map
fig.show()


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
## Narrow down cities that fit criteria and drop any results with null values
# Narrow down cities that fit criteria
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 25) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
print(filtered_cities)


     City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
18        18  yellowknife  62.4560 -114.3525    296.44        46          75   
41        41      lusambo  -4.9667   23.4500    301.12        35          99   
44        44   whitehorse  60.7161 -135.0538    298.20        30          20   
79        79  san ignacio  17.1561  -89.0714    308.23        49          40   
82        82      winslow  35.0242 -110.6974    312.25         6           0   
..       ...          ...      ...       ...       ...       ...         ...   
549      549    taoudenni  22.6783   -3.9836    310.07        29          44   
557      557     fiambala -27.6871  -67.6149    285.06        36          14   
560      560       verdun  49.2500    5.3333    291.02        49           6   
565      565     tshikapa  -6.4167   20.8000    297.57        31           8   
567      567   sioux city  42.5000  -96.4003    300.92        47          20   

     Wind Speed Country        Date  
1

### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# hotel_df['Hotel Name'] = ""

# Display sample data
# print(hotel_df)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel (in meters)
radius = 5000 
params = {
    "radius": radius,
    "type": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"point:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
ust'-kulom - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
gorzyce - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
kelibia - nearest hotel: No hotel found
saguenay - nearest hotel: No hotel found
grand falls-windsor - nearest hotel: No hotel found
forster - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
columbia - nearest hotel: No hotel found
egilsstadir - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
ebon - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
ardesen - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
callao - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ribeira grande,PT,38.5167,-28.7000,76,No hotel found
1,port saint john's,ZA,-31.6229,29.5448,60,No hotel found
2,ust'-kulom,RU,61.6864,53.6902,71,No hotel found
3,port-aux-francais,TF,-49.3500,70.2167,54,No hotel found
4,gorzyce,PL,50.6672,21.8401,85,No hotel found
...,...,...,...,...,...,...
564,paraiso,CR,9.8383,-83.8656,98,No hotel found
565,tshikapa,CD,-6.4167,20.8000,31,No hotel found
566,papao,PF,-17.7500,-149.5500,67,No hotel found
567,sioux city,US,42.5000,-96.4003,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:

# Configure the map plot
fig = px.scatter_mapbox(city_data_df, lat='Lat', lon='Lng', hover_name='City', size='Humidity',
                        color='Humidity', zoom=3)

# Display the map
fig.show()